In [9]:
from vnstock import *
import pandas as pd
ticket = ['ACB', 'BCM', 'BID', 'BVH', 'CTG']
frame = []
for i in ticket:
    a = stock_historical_data(symbol=i, 
                            start_date="2022-01-01", 
                            end_date='2022-01-11')    
    a["ticker"] = i
    frame.append(a)
    
q = pd.concat(frame)
close= q.pivot_table(values="Close", columns="ticker", index="TradingDate")
returns = close.pct_change()

In [21]:
r = returns.dropna().values

In [25]:
r

array([[-0.02034884,  0.00304841, -0.00935829, -0.00528416,  0.        ],
       [ 0.00148368,  0.06968646,  0.02564103, -0.00706414, -0.02158273],
       [-0.01333333,  0.06941549,  0.03157895, -0.01068109, -0.02352941],
       [-0.00900901, -0.06887483, -0.00892857, -0.01618502,  0.00301205],
       [-0.00757576,  0.06969675,  0.01415701, -0.01645128, -0.00900901]])

In [45]:
len(close)

6

In [52]:
import numpy as np
import pandas as pd
import random

# Load the time series data into a pandas DataFrame
data = close

# Define the Q-Learning algorithm
def q_learning(data, n_actions, alpha, gamma, epsilon, max_steps):
    # Initialize the Q-table with zeros
    Q = np.zeros((data.shape[0], n_actions))

    for i in range(max_steps):
        # Initialize the state
        state = np.random.randint(0, data.shape[0])
        
        # Choose an action using epsilon-greedy policy
        if random.uniform(0, 1) < epsilon:
            action = np.random.choice(n_actions)
        else:
            action = np.argmax(Q[state, :])

        next_state = state + 1
        # Get the next state and reward based on the action
        if next_state == len(close):
            next_state = np.random.randint(0,len(close))
    
        reward = data.iloc[next_state, action] - data.iloc[state, action]
        
        # Update the Q-table using Bellman equation
        Q[state, action] = (1 - alpha) * Q[state, action] + alpha * (reward + gamma * np.max(Q[next_state, :]))
    
    return Q

# Define the number of actions (i.e., the number of stocks to invest in)
n_actions = 5

# Define the learning rate, discount factor, and exploration rate
alpha = 0.1
gamma = 0.9
epsilon = 0.1

# Define the number of steps
max_steps = 1000

# Run the Q-Learning algorithm
Q = q_learning(data, n_actions, alpha, gamma, epsilon, max_steps)

df = pd.DataFrame(Q)
df.div(df.abs().sum(axis=1),axis=0)


In [54]:
close

ticker,ACB,BCM,BID,BVH,CTG
TradingDate,,,,,
2022-01-04,27520.0,65280.0,37400.0,53367.0,34750.0
2022-01-05,26960.0,65479.0,37050.0,53085.0,34750.0
2022-01-06,27000.0,70042.0,38000.0,52710.0,34000.0
2022-01-07,26640.0,74904.0,39200.0,52147.0,33200.0
2022-01-10,26400.0,69745.0,38850.0,51303.0,33300.0
2022-01-11,26200.0,74606.0,39400.0,50459.0,33000.0


,0,1,2,3,4
0,0.120154,0.464921,0.124451,0.151044,0.139429
1,0.487345,0.197156,0.105982,0.108744,0.100773
2,0.095467,0.803630,0.061139,0.036278,-0.003486
3,0.064287,0.104173,0.166649,0.087972,0.576919
4,0.118990,0.690785,0.021254,0.120340,0.048632
5,0.545629,0.027309,0.051111,0.172549,0.203401


In [64]:
df

,0,1,2,3,4
0,3662.570449,14171.847144,3793.539155,4604.168195,4250.119016
1,16114.349328,6519.077534,3504.365680,3595.680087,3332.127400
2,2165.563771,18229.412004,1386.855440,822.922520,-79.072212
3,1729.254912,2802.161918,4482.692488,2366.370709,15518.572332
4,3076.148397,17858.254942,549.458043,3111.039165,1257.233734
5,15318.615428,766.707965,1434.949042,4844.338310,5710.518967


0    30482.243959
1    33065.600030
2    22683.825947
3    26899.052359
4    25852.134282
5    28075.129711
dtype: float64

In [1]:
import sys
path = "Find Strategy"
sys.path.append(path)

from find_mix_strategy import *
from q_learning import *

In [8]:
a = find_best_strategy([[0,0,0],[3,-1,-1],[2,2,-2]],num_strategy=3,prob_rival=[1,0,0])

In [9]:
a.run()

Iteration 0, strategy 2 with Q value 0.2
Iteration 1, strategy 2 with Q value 0.38
Iteration 2, strategy 2 with Q value 0.542
Iteration 3, strategy 2 with Q value 0.6878000000000001
Iteration 4, strategy 0 with Q value 0.0
Iteration 5, strategy 1 with Q value 0.30000000000000004
Iteration 6, strategy 2 with Q value 0.8190200000000001
Iteration 7, strategy 2 with Q value 0.9371180000000001
Iteration 8, strategy 0 with Q value 0.0
Iteration 9, strategy 2 with Q value 1.0434062000000002
Iteration 10, strategy 2 with Q value 1.1390655800000002
Iteration 11, strategy 2 with Q value 1.2251590220000002
Iteration 12, strategy 2 with Q value 1.3026431198000001
Iteration 13, strategy 2 with Q value 1.37237880782
Iteration 14, strategy 2 with Q value 1.435140927038
Iteration 15, strategy 2 with Q value 1.4916268343342
Iteration 16, strategy 2 with Q value 1.5424641509007802
Iteration 17, strategy 2 with Q value 1.5882177358107021
Iteration 18, strategy 2 with Q value 1.629395962229632
Iteration 1

In [7]:
a.show_statistics()

Strategy 0 with k: 353
Strategy 1 with k: 4608
Strategy 2 with k: 5039


In [24]:
find_mix_strategy_in_zero_sum([[0,0,0],[3,-1,-1],[2,2,-2]])

'Kỳ vọng của mình: 0.0 \nChiến lược đối thủ: [0. 0. 1. 0.]'

In [23]:
print(find_mix_strategy([[0,0,0],[3,-1,-1],[2,2,-2]],[[0,3,2],[0,1,2],[0,-1,-2]]))

Chiến lược của người 1:[1. 0. 0.]
Chiến lược của người 2:[0. 0. 1.],
Kỳ vọng của người 1: [0.],
Kỳ vọng của người 2: [0.]


In [2]:
a = [[1,-2,-3],[-1,2,-3],[-1,-2,3]]
b = [[-1,2,3],[1,-2,3],[1,2,-3]]
print(find_mix_strategy(a,b))

Chiến lược của người 1:[0.54545455 0.27272727 0.18181818]
Chiến lược của người 2:[0.  0.6 0.4],
Kỳ vọng của người 1: [0.],
Kỳ vọng của người 2: [0.54545455]


In [2]:
a = [[6,3],[10,0]]
b = [[2,6],[-1,0]]
print(find_mix_strategy(a,b))

Chiến lược của người 1:[1. 0.]
Chiến lược của người 2:[0. 1.],
Kỳ vọng của người 1: [3.],
Kỳ vọng của người 2: [2.]


In [3]:
a = [[0,2],[1,0]]
b = [[0,1],[2,0]]
print(find_mix_strategy(a,b))

Chiến lược của người 1:[0.33333333 0.66666667]
Chiến lược của người 2:[0.66666667 0.33333333],
Kỳ vọng của người 1: [0.66666667],
Kỳ vọng của người 2: [0.66666667]


In [4]:
a = [[0,-1,1],[1,0,-1],[-1,1,0]]
b = [[0,1,-1],[-1,0,1],[1,-1,0]]
print(find_mix_strategy(a,b))

Chiến lược của người 1:[0.33333333 0.33333333 0.33333333]
Chiến lược của người 2:[0.33333333 0.33333333 0.33333333],
Kỳ vọng của người 1: [0.],
Kỳ vọng của người 2: [0.]


In [6]:
payoff1 = [[2,1,3],[3,2,1],[1,3,2]]
payoff2 = [[2,3,1],[1,2,3],[3,1,2]]
print(find_mix_strategy(payoff1,payoff2))

Chiến lược của người 1:[0.33333333 0.33333333 0.33333333]
Chiến lược của người 2:[0.33333333 0.33333333 0.33333333],
Kỳ vọng của người 1: [2.],
Kỳ vọng của người 2: [2.]


In [44]:

find_mix_strategy([[2,0],[0,1]])

'Kỳ vọng của mình: 0.6666666666666666 \nChiến lược đối thủ: [0.33333333 0.66666667 0.66666667]'